# Proyecto 1: Analítica de textos

## Miembros
- Estudainte 3: Mariana Pineda Miranda - 202123330
- Estudiante 2: Isaac David Bermúdez- 202014146
- Estudiante 1: Joseph Linares Gutierrez - 202111887

In [26]:
import re
import numpy as np
import pandas as pd

from joblib import dump, load

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import scipy.stats as stats

pd.set_option('display.max_columns', None)   # todas las columnas
pd.set_option('display.width', None) 

## Perfilamiento y análisis de datos

In [27]:
dataTrain = pd.read_csv("data/train.csv")
dataTest = pd.read_csv("data/test.csv")

In [28]:
dataTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2424 entries, 0 to 2423
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   textos  2424 non-null   object
 1   labels  2424 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 38.0+ KB


In [29]:
dataTrain['textos'].value_counts()

"Aprendizaje" y "educación" se consideran sinónimos de escolarización formal. Las organizaciones auxiliares, como las editoriales de educación, las juntas examinadoras y las organizaciones de formación de docentes, se consideran extensiones de los acuerdos establecidos por los gobiernos. Este marco de comprensión se ha vuelto cada vez más inadecuado.                                                                                                                                                                                                                                                                                                                                                                                                                       1
El arrendamiento de grandes porciones de estas tierras consuetudinarias a propietarios privados afecta a la seguridad alimentaria y a todas las actividades de subsistencia de quienes antes dependían de estos recursos naturales. Los despose

In [30]:
dataTrain['labels'].value_counts()

4    1025
3     894
1     505
Name: labels, dtype: int64

In [31]:
dataTrain['textos'].str.len().describe()

count    2424.000000
mean      683.223185
std       223.563695
min       303.000000
25%       513.000000
50%       647.000000
75%       809.250000
max      1492.000000
Name: textos, dtype: float64

In [32]:
dataTrain.isnull().sum()

textos    0
labels    0
dtype: int64

## Tratamiento de Datos

#### Descarga de las stopwords en español

In [58]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\josep\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [59]:
from nltk.corpus import stopwords

# Lista de stopwords en español
stopwordsInSpanish = stopwords.words('spanish')

print(stopwordsInSpanish)

['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin', 'sobre', 'también', 'me', 'hasta', 'hay', 'donde', 'quien', 'desde', 'todo', 'nos', 'durante', 'todos', 'uno', 'les', 'ni', 'contra', 'otros', 'ese', 'eso', 'ante', 'ellos', 'e', 'esto', 'mí', 'antes', 'algunos', 'qué', 'unos', 'yo', 'otro', 'otras', 'otra', 'él', 'tanto', 'esa', 'estos', 'mucho', 'quienes', 'nada', 'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas', 'algo', 'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus', 'ellas', 'nosotras', 'vosotros', 'vosotras', 'os', 'mío', 'mía', 'míos', 'mías', 'tuyo', 'tuya', 'tuyos', 'tuyas', 'suyo', 'suya', 'suyos', 'suyas', 'nuestro', 'nuestra', 'nuestros', 'nuestras', 'vuestro', 'vuestra', 'vuestros', 'vuestras', 'esos', 'esas', 'estoy', 'estás', 'está', 'estamos', 'estáis', 'están', 'e

#### Limpieza y vectorización de datos

In [ ]:
def limpiar_texto(texto: str) -> str:
    texto = texto.lower()                               # Convertir a minusculas
    texto = re.sub(r'[^a-záéíóúüñ\s]', '', texto)       # Eliminar caracteres especiales y numeros
    return texto

dataTrain['texto_limpio'] = dataTrain['textos'].apply(limpiar_texto)

vectorizer = TfidfVectorizer(stop_words = stopwordsInSpanish, max_features=1000)
X = vectorizer.fit_transform(dataTrain['texto_limpio'])
y = dataTrain['labels']

In [76]:
print(dataTrain.sample(5))


                                                 textos  labels  \
1922  Una mayor participación en dichos programas pu...       4   
125   La OCDE está elaborando mejores datos sobre lo...       3   
1889  Esta estrategia enfatiza la necesidad de la pr...       3   
2013  Por lo tanto, la generalización de estos estud...       3   
302   Sin embargo, la ausencia de presupuestos pluri...       4   

                                           texto_limpio  
1922  una mayor participación en dichos programas pu...  
125   la ocde está elaborando mejores datos sobre lo...  
1889  esta estrategia enfatiza la necesidad de la pr...  
2013  por lo tanto la generalización de estos estudi...  
302   sin embargo la ausencia de presupuestos pluria...  
